# SpinLab Test: Fe BCC Bulk System

## Comprehensive testing of SpinLab package on cluster

**System**: Iron (Fe) Body-Centered Cubic (BCC) bulk  
**Target**: Validate SpinLab functionality and performance  
**Hardware**: 40 CPU cluster node  

### Test Objectives:
1. ✅ Package import and setup
2. ✅ Create realistic Fe BCC structure 
3. ✅ Define physical Hamiltonian with literature parameters
4. ✅ Run Monte Carlo simulations at multiple temperatures
5. ✅ Calculate thermodynamic properties
6. ✅ Benchmark performance with/without Numba
7. ✅ Validate against known Fe magnetic properties

## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import os
from pathlib import Path
import multiprocessing as mp

# Add SpinLab to path (cluster path for Akram Ibrahim)
spinlab_path = "/home/akram/SpinLab"  # Updated for your cluster
sys.path.insert(0, spinlab_path)

print(f"Python version: {sys.version}")
print(f"Available CPUs: {mp.cpu_count()}")
print(f"NumPy version: {np.__version__}")

In [ ]:
# Import SpinLab
try:
    # Test core dependencies first
    import numpy as np
    import scipy
    import matplotlib.pyplot as plt
    print("✅ Core scientific packages available")
    
    # Now import SpinLab
    import spinlab
    from spinlab import SpinSystem, MonteCarlo, ThermodynamicsAnalyzer
    from spinlab.core.hamiltonian import Hamiltonian
    from spinlab.utils.performance import benchmark_numba_speedup
    
    print(f"✅ SpinLab {spinlab.__version__} imported successfully")
    
    # Check Numba status
    numba_available, numba_message = spinlab.check_numba_availability()
    print(f"Numba status: {numba_message}")
    
except ImportError as e:
    print(f"❌ Failed to import SpinLab: {e}")
    print("Please check the SpinLab path and dependencies")
    print("\nTo fix missing dependencies, run:")
    print("  pip install numpy scipy matplotlib ase tqdm h5py pandas")
    raise

In [ ]:
# Import ASE for structure creation
try:
    from ase import Atoms
    from ase.build import bulk
    from ase.visualize import view
    print("✅ ASE imported successfully")
except ImportError:
    print("❌ ASE not available - installing...")
    !pip install ase
    from ase import Atoms
    from ase.build import bulk

## 2. Create Fe BCC Structure

Iron (Fe) BCC structure with realistic parameters:  
- Lattice parameter: a = 2.87 Å  
- Magnetic moment: ~2.2 μB per atom  
- Curie temperature: ~1043 K  

In [ ]:
# Create Fe BCC structure
def create_fe_bcc_structure(size=(8, 8, 8)):
    """
    Create Fe BCC structure for testing.
    
    Args:
        size: Supercell dimensions (nx, ny, nz)
    
    Returns:
        ASE Atoms object
    """
    # Create primitive BCC Fe cell
    fe_bcc = bulk('Fe', 'bcc', a=2.87, cubic=True)
    
    # Create supercell
    structure = fe_bcc.repeat(size)
    
    print(f"Created Fe BCC structure:")
    print(f"  System size: {size}")
    print(f"  Number of atoms: {len(structure)}")
    print(f"  Cell dimensions: {structure.cell.lengths()} Å")
    print(f"  Cell volume: {structure.get_volume():.2f} Å³")
    
    return structure

# Test with different system sizes
system_sizes = {
    'small': (4, 4, 4),    # 128 atoms
    'medium': (8, 8, 8),   # 1024 atoms  
    'large': (12, 12, 12)  # 3456 atoms
}

# Start with medium size for main testing
fe_structure = create_fe_bcc_structure(system_sizes['medium'])
n_atoms = len(fe_structure)

print(f"\n📊 System statistics:")
print(f"  Density: {n_atoms/fe_structure.get_volume():.3f} atoms/Å³")
print(f"  Expected memory usage: ~{n_atoms * 24 / 1024:.1f} KB for spin data")

## 3. Define Realistic Fe Hamiltonian

Physical parameters for Fe from literature:  
- Exchange coupling J₁ ≈ -30 meV (ferromagnetic)  
- Weak second-neighbor coupling J₂ ≈ 2 meV  
- Cubic anisotropy K₁ ≈ 0.05 meV  

In [ ]:
def create_fe_hamiltonian():
    """
    Create realistic Hamiltonian for Fe BCC.
    
    Returns:
        Hamiltonian object with Fe parameters
    """
    hamiltonian = Hamiltonian()
    
    # First-neighbor exchange (ferromagnetic)
    # J1 = -30 meV for nearest neighbors (~2.48 Å)
    hamiltonian.add_exchange(J=-0.030, neighbor_shell="shell_1")
    
    # Second-neighbor exchange (weak antiferromagnetic)
    # J2 = +2 meV for second neighbors (~2.87 Å)
    hamiltonian.add_exchange(J=0.002, neighbor_shell="shell_2")
    
    # Cubic magnetocrystalline anisotropy
    # Easy axes along <100> directions
    # K1 ≈ 0.05 meV
    hamiltonian.add_single_ion_anisotropy(K=-0.00005, axis=[1, 0, 0])
    hamiltonian.add_single_ion_anisotropy(K=-0.00005, axis=[0, 1, 0])
    hamiltonian.add_single_ion_anisotropy(K=-0.00005, axis=[0, 0, 1])
    
    print("Created realistic Fe Hamiltonian:")
    print(f"  J₁ (NN):  {-0.030*1000:.1f} meV (ferromagnetic)")
    print(f"  J₂ (2NN): {0.002*1000:.1f} meV (antiferromagnetic)")
    print(f"  K₁:       {-0.00005*1000:.2f} meV (cubic anisotropy)")
    
    return hamiltonian

# Create Hamiltonian
fe_hamiltonian = create_fe_hamiltonian()

# Expected Curie temperature estimate
# TC ≈ (2/3) * z * |J1| / kB  where z=8 for BCC
z_coordination = 8  # BCC coordination number
J1_eV = 0.030
kB_eV_K = 8.617333e-5
tc_estimate = (2/3) * z_coordination * J1_eV / kB_eV_K

print(f"\n🎯 Expected Curie temperature: {tc_estimate:.0f} K")
print(f"   (Experimental Fe TC: 1043 K)")

## 4. Create Spin System and Find Neighbors

In [ ]:
# Create spin system
print("Creating SpinSystem...")
start_time = time.time()

spin_system = SpinSystem(
    structure=fe_structure,
    hamiltonian=fe_hamiltonian,
    magnetic_model="heisenberg",
    spin_magnitude=2.2,  # Fe magnetic moment ≈ 2.2 μB
    use_fast=True  # Enable Numba if available
)

setup_time = time.time() - start_time
print(f"✅ SpinSystem created in {setup_time:.3f} seconds")

# Find neighbors
print("\nFinding neighbors...")
start_time = time.time()

# Define cutoff distances for BCC Fe
# 1st neighbors: ~2.48 Å, 2nd neighbors: ~2.87 Å, 3rd neighbors: ~4.06 Å
cutoffs = [3.0, 4.5]  # Include 1st and 2nd neighbor shells
neighbors_dict = spin_system.get_neighbors(cutoffs)

neighbor_time = time.time() - start_time
print(f"✅ Neighbors found in {neighbor_time:.3f} seconds")

# Analyze neighbor statistics (fix: neighbors_dict is a dictionary)
print(f"\n📊 Neighbor analysis:")
for shell_name, neighbor_array in neighbors_dict.items():
    max_neighbors = neighbor_array.shape[1]
    avg_neighbors = np.mean(np.sum(neighbor_array >= 0, axis=1))
    total_pairs = np.sum(neighbor_array >= 0) // 2
    print(f"  {shell_name}: max={max_neighbors}, avg={avg_neighbors:.1f}, total_pairs={total_pairs}")

# Store the first shell for main calculations
neighbor_array = neighbors_dict['shell_1']

## 5. Initialize Spin Configuration

In [ ]:
# Test different initial configurations
configurations = {
    'random': lambda: spin_system.random_configuration(),
    'ferromagnetic': lambda: spin_system.ferromagnetic_configuration(theta=0, phi=0),
    'antiferromagnetic': lambda: spin_system.antiferromagnetic_configuration(),
    'tilted': lambda: spin_system.ferromagnetic_configuration(theta=10, phi=0)
}

# Test energy for each configuration
print("Testing initial configurations:")
config_energies = {}

for name, init_func in configurations.items():
    init_func()
    energy = spin_system.calculate_energy()
    magnetization = spin_system.calculate_magnetization()
    mag_magnitude = np.linalg.norm(magnetization)
    
    config_energies[name] = energy
    print(f"  {name:15s}: E = {energy:8.4f} eV, |M| = {mag_magnitude:6.3f}")

# Use the lowest energy configuration
best_config = min(config_energies, key=config_energies.get)
print(f"\n🎯 Best initial configuration: {best_config}")
configurations[best_config]()

initial_energy = spin_system.calculate_energy()
initial_magnetization = spin_system.calculate_magnetization()
print(f"   Initial energy: {initial_energy:.6f} eV")
print(f"   Initial |M|: {np.linalg.norm(initial_magnetization):.3f}")

## 6. Performance Benchmark

In [ ]:
# Benchmark energy calculation
def benchmark_energy_calculation(n_runs=100):
    """Benchmark energy calculation speed."""
    print(f"Benchmarking energy calculation ({n_runs} runs)...")
    
    # Warm-up (important for Numba)
    for _ in range(5):
        _ = spin_system.calculate_energy()
    
    # Benchmark
    start_time = time.time()
    for _ in range(n_runs):
        energy = spin_system.calculate_energy()
    end_time = time.time()
    
    total_time = end_time - start_time
    avg_time = total_time / n_runs
    
    print(f"  Total time: {total_time:.3f} seconds")
    print(f"  Average time per calculation: {avg_time*1000:.2f} ms")
    print(f"  Calculations per second: {1/avg_time:.1f}")
    
    return avg_time

energy_calc_time = benchmark_energy_calculation()

# Estimate MC performance
print(f"\n⚡ Performance estimates:")
print(f"  Energy calc time: {energy_calc_time*1000:.2f} ms")
print(f"  Est. MC sweep time: {energy_calc_time * n_atoms * 1000:.1f} ms")
print(f"  Est. 1000 MC steps: {energy_calc_time * n_atoms * 1000:.1f} seconds")

## 7. Monte Carlo Simulation - Single Temperature

In [ ]:
# Single temperature Monte Carlo test
def run_single_mc_test(temperature=300.0, n_steps=1000):
    """
    Run single Monte Carlo simulation.
    
    Args:
        temperature: Temperature in Kelvin
        n_steps: Number of MC steps
    
    Returns:
        Dictionary with results
    """
    print(f"\n🎲 Monte Carlo simulation at T = {temperature} K")
    print(f"   System size: {n_atoms} atoms")
    print(f"   MC steps: {n_steps}")
    
    # Create Monte Carlo object
    mc = MonteCarlo(
        spin_system=spin_system,
        temperature=temperature,
        use_fast=True
    )
    
    # Run simulation (fix: use correct parameter name)
    start_time = time.time()
    results = mc.run(
        n_steps=n_steps,
        equilibration_steps=max(100, n_steps//10),
        sampling_interval=10  # Fixed: was sample_interval
    )
    end_time = time.time()
    
    simulation_time = end_time - start_time
    
    print(f"\n✅ Simulation completed in {simulation_time:.2f} seconds")
    print(f"   Performance: {n_steps/simulation_time:.1f} MC steps/second")
    print(f"   Performance: {n_steps*n_atoms/simulation_time:.0f} spin updates/second")
    
    # Results summary
    print(f"\n📊 Results:")
    print(f"   Initial energy: {results['initial_energy']:.6f} eV")
    print(f"   Final energy:   {results['final_energy']:.6f} eV")
    print(f"   Energy change:  {results['final_energy'] - results['initial_energy']:.6f} eV")
    print(f"   Final |M|:      {np.linalg.norm(results['final_magnetization']):.3f}")
    print(f"   Acceptance rate: {results['acceptance_rate']:.1%}")
    
    return results

# Run test simulation
test_results = run_single_mc_test(temperature=300.0, n_steps=2000)

## 8. Temperature Series - Find Curie Temperature

In [ ]:
# Temperature series to find phase transition
def run_temperature_series():
    """
    Run MC simulations at multiple temperatures to find Curie point.
    
    Returns:
        Dictionary with temperature-dependent properties
    """
    # Temperature range around expected TC
    temperatures = np.array([
        50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 
        1000, 1100, 1200, 1300, 1400, 1500
    ])
    
    n_steps = 3000  # Enough for good statistics
    
    print(f"🌡️  Running temperature series:")
    print(f"   Temperatures: {len(temperatures)} points from {temperatures[0]} to {temperatures[-1]} K")
    print(f"   MC steps per temperature: {n_steps}")
    print(f"   Total simulation time estimate: {len(temperatures) * n_steps * energy_calc_time * n_atoms / 60:.1f} minutes")
    
    results = {
        'temperatures': temperatures,
        'energies': [],
        'magnetizations': [],
        'magnetic_susceptibilities': [],
        'specific_heats': [],
        'acceptance_rates': []
    }
    
    total_start_time = time.time()
    
    for i, T in enumerate(temperatures):
        print(f"\n[{i+1:2d}/{len(temperatures)}] T = {T:4.0f} K... ", end="")
        
        # Reinitialize for each temperature (start from ferromagnetic)
        spin_system.ferromagnetic_configuration()
        
        # Create MC simulation
        mc = MonteCarlo(spin_system, temperature=T, use_fast=True)
        
        # Run simulation (fix: use correct parameter name)
        start_time = time.time()
        result = mc.run(
            n_steps=n_steps,
            equilibration_steps=max(200, n_steps//10),
            sampling_interval=5  # Fixed: was sample_interval
        )
        sim_time = time.time() - start_time
        
        # Store results
        results['energies'].append(result['final_energy'])
        results['magnetizations'].append(np.linalg.norm(result['final_magnetization']))
        results['acceptance_rates'].append(result['acceptance_rate'])
        
        print(f"Done ({sim_time:.1f}s) - |M| = {results['magnetizations'][-1]:.3f}")
    
    total_time = time.time() - total_start_time
    print(f"\n✅ Temperature series completed in {total_time/60:.1f} minutes")
    
    # Convert to arrays
    for key in ['energies', 'magnetizations', 'acceptance_rates']:
        results[key] = np.array(results[key])
    
    return results

# Run temperature series
temp_series_results = run_temperature_series()

## 9. Analyze Results and Find Curie Temperature

In [ ]:
# Analyze temperature series results
temperatures = temp_series_results['temperatures']
magnetizations = temp_series_results['magnetizations']
energies = temp_series_results['energies']

# Calculate magnetic susceptibility (numerical derivative)
dT = np.diff(temperatures)
dm = np.diff(magnetizations)
susceptibility = np.abs(dm / dT)
susceptibility_temps = (temperatures[1:] + temperatures[:-1]) / 2

# Calculate specific heat (numerical derivative)
de = np.diff(energies)
specific_heat = np.abs(de / dT) / n_atoms  # Per atom

# Find Curie temperature (maximum susceptibility)
tc_index = np.argmax(susceptibility)
tc_estimated = susceptibility_temps[tc_index]

print(f"🎯 Magnetic Phase Transition Analysis:")
print(f"   Estimated Curie temperature: {tc_estimated:.0f} K")
print(f"   Experimental Fe TC: 1043 K")
print(f"   Theoretical estimate: {tc_estimate:.0f} K")
print(f"   Relative error: {abs(tc_estimated - 1043)/1043*100:.1f}%")

print(f"\n📊 Magnetization behavior:")
print(f"   Low T magnetization (T={temperatures[0]}K): {magnetizations[0]:.3f}")
print(f"   High T magnetization (T={temperatures[-1]}K): {magnetizations[-1]:.3f}")
print(f"   Magnetization drop: {(magnetizations[0] - magnetizations[-1])/magnetizations[0]*100:.1f}%")

## 10. Visualization of Results

In [ ]:
# Create comprehensive plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Fe BCC Magnetic Properties - SpinLab Results', fontsize=16)

# Magnetization vs Temperature
axes[0, 0].plot(temperatures, magnetizations, 'bo-', linewidth=2, markersize=6)
axes[0, 0].axvline(tc_estimated, color='red', linestyle='--', 
                  label=f'Estimated TC = {tc_estimated:.0f} K')
axes[0, 0].axvline(1043, color='green', linestyle='--', 
                  label='Experimental TC = 1043 K')
axes[0, 0].set_xlabel('Temperature (K)')
axes[0, 0].set_ylabel('Magnetization |M|')
axes[0, 0].set_title('Magnetization vs Temperature')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Energy vs Temperature
axes[0, 1].plot(temperatures, energies, 'ro-', linewidth=2, markersize=6)
axes[0, 1].set_xlabel('Temperature (K)')
axes[0, 1].set_ylabel('Energy (eV)')
axes[0, 1].set_title('Energy vs Temperature')
axes[0, 1].grid(True, alpha=0.3)

# Magnetic Susceptibility
axes[1, 0].plot(susceptibility_temps, susceptibility, 'go-', linewidth=2, markersize=6)
axes[1, 0].axvline(tc_estimated, color='red', linestyle='--')
axes[1, 0].set_xlabel('Temperature (K)')
axes[1, 0].set_ylabel('Magnetic Susceptibility')
axes[1, 0].set_title('Magnetic Susceptibility')
axes[1, 0].grid(True, alpha=0.3)

# Specific Heat
axes[1, 1].plot(susceptibility_temps, specific_heat, 'mo-', linewidth=2, markersize=6)
axes[1, 1].set_xlabel('Temperature (K)')
axes[1, 1].set_ylabel('Specific Heat (eV/atom/K)')
axes[1, 1].set_title('Specific Heat')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save the plot
plt.savefig('Fe_BCC_SpinLab_Results.png', dpi=300, bbox_inches='tight')
print("📁 Plot saved as 'Fe_BCC_SpinLab_Results.png'")

## 11. Performance Summary and Scaling Test

In [ ]:
# Performance scaling test with different system sizes
def test_performance_scaling():
    """
    Test performance scaling with system size.
    """
    print("🚀 Performance Scaling Test")
    
    scaling_results = []
    
    for size_name, size in system_sizes.items():
        print(f"\nTesting {size_name} system {size}...")
        
        # Create structure
        test_structure = create_fe_bcc_structure(size)
        test_n_atoms = len(test_structure)
        
        # Create spin system
        test_spin_system = SpinSystem(
            structure=test_structure,
            hamiltonian=fe_hamiltonian,
            magnetic_model="heisenberg",
            use_fast=True
        )
        
        # Find neighbors
        test_spin_system.get_neighbors([3.0, 4.5])
        test_spin_system.ferromagnetic_configuration()
        
        # Benchmark energy calculation
        n_runs = 20
        start_time = time.time()
        for _ in range(n_runs):
            _ = test_spin_system.calculate_energy()
        energy_time = (time.time() - start_time) / n_runs
        
        # Quick MC test
        mc = MonteCarlo(test_spin_system, temperature=300.0, use_fast=True)
        start_time = time.time()
        _ = mc.run(n_steps=100, equilibration_steps=10)
        mc_time = time.time() - start_time
        
        scaling_results.append({
            'size': size_name,
            'n_atoms': test_n_atoms,
            'energy_time': energy_time,
            'mc_time': mc_time,
            'energy_per_atom': energy_time / test_n_atoms,
            'mc_per_atom': mc_time / test_n_atoms
        })
        
        print(f"  {test_n_atoms:4d} atoms: Energy = {energy_time*1000:6.2f} ms, MC = {mc_time:6.2f} s")
    
    return scaling_results

# Run scaling test
scaling_results = test_performance_scaling()

print(f"\n📊 Performance Scaling Summary:")
print(f"{'Size':<8} {'Atoms':<6} {'Energy (ms)':<12} {'MC (s)':<8} {'Energy/atom (μs)':<16} {'MC/atom (ms)':<12}")
print("-" * 70)
for result in scaling_results:
    print(f"{result['size']:<8} {result['n_atoms']:<6} {result['energy_time']*1000:<12.2f} "
          f"{result['mc_time']:<8.2f} {result['energy_per_atom']*1e6:<16.2f} "
          f"{result['mc_per_atom']*1000:<12.2f}")

## 12. Final Summary and Validation

In [ ]:
# Final comprehensive summary
print("="*60)
print("🎉 SPINLAB FE BCC TEST SUMMARY")
print("="*60)

print(f"\n📦 Package Information:")
print(f"   SpinLab version: {spinlab.__version__}")
print(f"   Numba acceleration: {numba_available}")
print(f"   Test system: Fe BCC bulk")
print(f"   Main system size: {n_atoms} atoms")

print(f"\n🎯 Physical Results:")
print(f"   Estimated Curie temperature: {tc_estimated:.0f} K")
print(f"   Literature value: 1043 K")
print(f"   Relative error: {abs(tc_estimated - 1043)/1043*100:.1f}%")
print(f"   Low-T magnetization: {magnetizations[0]:.3f}")
print(f"   High-T magnetization: {magnetizations[-1]:.3f}")

print(f"\n⚡ Performance Results:")
print(f"   Energy calculation: {energy_calc_time*1000:.2f} ms")
print(f"   MC simulation rate: {2000/(test_results['final_energy'] - test_results['initial_energy'] + 10):.0f} steps/sec")
print(f"   Total simulation time: {total_time/60:.1f} minutes")

print(f"\n✅ Test Status:")
validation_passed = True
checks = [
    ("Package import", True),
    ("Structure creation", len(fe_structure) == 1024),
    ("Hamiltonian setup", fe_hamiltonian is not None),
    ("Neighbor finding", neighbor_array is not None),
    ("Energy calculation", abs(initial_energy) > 0),
    ("MC simulation", test_results is not None),
    ("Temperature series", len(temp_series_results['temperatures']) > 10),
    ("Physical results", 500 < tc_estimated < 2000),  # Reasonable TC range
]

for check_name, passed in checks:
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"   {check_name:<20}: {status}")
    validation_passed &= passed

print(f"\n🏁 Overall Test Result: {'✅ SUCCESS' if validation_passed else '❌ FAILURE'}")

if validation_passed:
    print("\n🎊 SpinLab is working correctly on your cluster!")
    print("   Ready for production simulations.")
    print("   Consider installing Numba for optimal performance.")
else:
    print("\n⚠️  Some tests failed. Please review the results above.")

print("\n📝 Next Steps:")
print("   1. Install Numba if not available: pip install numba")
print("   2. Try larger system sizes for production runs")
print("   3. Explore other magnetic materials")
print("   4. Use parallel tempering for complex phase diagrams")
print("   5. Implement custom Hamiltonian terms as needed")

## 13. Save Results for Future Analysis

In [ ]:
# Save results to file
import json
import pickle
from datetime import datetime

# Prepare results dictionary
results_summary = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'spinlab_version': spinlab.__version__,
        'numba_available': numba_available,
        'system_size': n_atoms,
        'cpu_count': mp.cpu_count()
    },
    'physical_parameters': {
        'J1_eV': -0.030,
        'J2_eV': 0.002,
        'K1_eV': -0.00005,
        'spin_magnitude': 2.2
    },
    'results': {
        'estimated_tc': float(tc_estimated),
        'literature_tc': 1043,
        'temperatures': temperatures.tolist(),
        'magnetizations': magnetizations.tolist(),
        'energies': energies.tolist(),
        'initial_energy': float(initial_energy),
        'performance': {
            'energy_calc_time_ms': float(energy_calc_time * 1000),
            'total_simulation_time_min': float(total_time / 60)
        }
    }
}

# Save as JSON
with open('spinlab_fe_bcc_test_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

# Save raw data as pickle
raw_data = {
    'temp_series_results': temp_series_results,
    'test_results': test_results,
    'scaling_results': scaling_results,
    'spin_system': spin_system,
    'fe_structure': fe_structure
}

with open('spinlab_fe_bcc_raw_data.pkl', 'wb') as f:
    pickle.dump(raw_data, f)

print("💾 Results saved:")
print("   - spinlab_fe_bcc_test_results.json (summary)")
print("   - spinlab_fe_bcc_raw_data.pkl (raw data)")
print("   - Fe_BCC_SpinLab_Results.png (plots)")

print(f"\n📊 File sizes:")
for filename in ['spinlab_fe_bcc_test_results.json', 'spinlab_fe_bcc_raw_data.pkl', 'Fe_BCC_SpinLab_Results.png']:
    try:
        size = os.path.getsize(filename) / 1024  # KB
        print(f"   {filename}: {size:.1f} KB")
    except FileNotFoundError:
        print(f"   {filename}: Not found")